# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-21c23e9843-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Ancient Civilization explorative search") 

Consider the following exploratory information need:

> Investigate the Ancient Egypt and the related archeological explorations and findings.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q13444093`| Meidum Pyramid    | node |
| `wd:Q41127`| Poseidon   | node |





Also consider

```
wd:Q13444093 ?p ?obj .
```

is the BGP to retrieve all **properties of Meidum Pyramid**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for Ancient Egypt

2. Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

3. Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

4. Identify the BGP for archeologist.

5. For each ancient civilization find the number of archeologists who worked on it (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

6. Find the BGP for deity.

7. List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

8. Compare ancient Greece with ancient Egypt and find how many deities each one has.


## Task 1
Identify the BGP for Ancient Egypt

In [3]:
# Let's get all properties and related object linked to Meidum Pyramid

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname
WHERE {
   wd:Q13444093 ?p ?obj. #MeidumPyramid

   ?p sc:name ?Pname.
   ?obj sc:name ?Oname.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P5008'), ('Pname', 'on focus list of Wikimedia project'), ('obj', 'http://www.wikidata.org/entity/Q68074438'), ('Oname', 'WikiProject Livius')]
[('p', 'http://www.wikidata.org/prop/direct/P88'), ('Pname', 'commissioned by'), ('obj', 'http://www.wikidata.org/entity/Q189371'), ('Oname', 'Sneferu')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('Pname', 'location'), ('obj', 'http://www.wikidata.org/entity/Q1066424'), ('Oname', 'Meidum')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('Pname', 'named after'), ('obj', 'http://www.wikidata.org/entity/Q1066424'), ('Oname', 'Meidum')]
[('p', 'http://www.wikidata.org/prop/direct/P88'), ('Pname', 'commissioned by'), ('obj', 'http://www.wikidata.org/entity/Q220240'), ('Oname', 'Huni')]
[('p', 'http://www.wikidata.org/prop/direct/P186'), ('Pname', 'made from material'), ('obj', 'http://www.wikidata.org/entity/Q23757'), ('Oname', 'limestone')]
[('p', 'http://www.wikidata.org/prop/dire

In [4]:
# Let's now get objects type names that are related to Meidum Pyramid, through one of the above found properties

queryString = """
SELECT DISTINCT ?Otype ?Oname count(*) as ?count
WHERE {
   wd:Q13444093 ?p ?o . #MeidumPyramid
   ?o wdt:P31 ?Otype .
   
   ?Otype sc:name ?Oname.
}

LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('Otype', 'http://www.wikidata.org/entity/Q6256'), ('Oname', 'country'), ('count', '1')]
[('Otype', 'http://www.wikidata.org/entity/Q3624078'), ('Oname', 'sovereign state'), ('count', '1')]
[('Otype', 'http://www.wikidata.org/entity/Q839954'), ('Oname', 'archaeological site'), ('count', '2')]
[('Otype', 'http://www.wikidata.org/entity/Q5'), ('Oname', 'human'), ('count', '2')]
[('Otype', 'http://www.wikidata.org/entity/Q1323642'), ('Oname', 'transcontinental country'), ('count', '1')]
[('Otype', 'http://www.wikidata.org/entity/Q204910'), ('Oname', 'governorate of Egypt'), ('count', '1')]
[('Otype', 'http://www.wikidata.org/entity/Q51576574'), ('Oname', 'Mediterranean country'), ('count', '1')]
[('Otype', 'http://www.wikidata.org/entity/Q532'), ('Oname', 'village'), ('count', '2')]
[('Otype', 'http://www.wikidata.org/entity/Q16695773'), ('Oname', 'WikiProject'), ('count', '1')]
[('Otype', 'http://www.wikidata.org/entity/Q4167836'), ('Oname', 'Wikimedia category'), ('count', '1')

In [5]:
#Let's get Egypt URI, could be useful for later

queryString = """
SELECT DISTINCT ?country ?countryName
WHERE {
    wd:Q13444093 wdt:P17 ?country. #MeidumPyramid country

    ?country sc:name ?countryName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q79'), ('countryName', 'Egypt')]
1


In [6]:
# Let's get objects related to Egypt and filter the classes names intuitively with regex

queryString = """
SELECT DISTINCT ?Otype ?Oname ?p ?Pname
WHERE {
   
   wd:Q79 ?p ?o . #Egypt
   ?o wdt:P31 ?Otype . #a
   
   ?Otype sc:name ?Oname.
   ?p sc:name ?Pname.
   FILTER(REGEX(?Oname,"(period|era|histor|cultur|civiliz)"))
}

LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('Otype', 'http://www.wikidata.org/entity/Q645883'), ('Oname', 'military operation'), ('p', 'http://www.wikidata.org/prop/direct/P463'), ('Pname', 'member of')]
[('Otype', 'http://www.wikidata.org/entity/Q99541706'), ('Oname', 'historical unrecognized state'), ('p', 'http://www.wikidata.org/prop/direct/P530'), ('Pname', 'diplomatic relation')]
[('Otype', 'http://www.wikidata.org/entity/Q3024240'), ('Oname', 'historical country'), ('p', 'http://www.wikidata.org/prop/direct/P1365'), ('Pname', 'replaces')]
[('Otype', 'http://www.wikidata.org/entity/Q3502482'), ('Oname', 'cultural region'), ('p', 'http://www.wikidata.org/prop/direct/P361'), ('Pname', 'part of')]
[('Otype', 'http://www.wikidata.org/entity/Q43702'), ('Oname', 'federation'), ('p', 'http://www.wikidata.org/prop/direct/P530'), ('Pname', 'diplomatic relation')]
[('Otype', 'http://www.wikidata.org/entity/Q512187'), ('Oname', 'federal republic'), ('p', 'http://www.wikidata.org/prop/direct/P530'), ('Pname', 'diplomatic rel

In [7]:
#"historical country" looks interesting
#Let's see what are the historical countries replaced by Egypt

queryString = """
SELECT DISTINCT ?histC ?Hname
WHERE {
    wd:Q13444093 ?p ?obj . #MeidumPyramid
    ?obj wdt:P17 wd:Q79; #country Egypt
         wdt:P1365 ?histC. #replaces
         
    ?histC sc:name ?Hname.
}

LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('histC', 'http://www.wikidata.org/entity/Q124943'), ('Hname', 'Kingdom of Egypt')]
[('histC', 'http://www.wikidata.org/entity/Q2051582'), ('Hname', 'All-Palestine Government')]
2


In [8]:
#Not promising result.
# Let's check if the historical country class has Ancient Egypt as an instance

queryString = """
SELECT DISTINCT ?histC ?histCname
WHERE {

    ?histC wdt:P31 wd:Q3024240 . #a HistoricalCountry

    ?histC sc:name ?histCname.
    FILTER(REGEX(?histCname,"Egypt"))
}

LIMIT 50
"""

print("Results")
x=run_query(queryString)

#It does, but it looks difficult to access it starting from the MeidumPyramid BGP
#As I don't know if it is a valid solution, I will try to access it directly from MeidumPyramid BGP

Results
[('histC', 'http://www.wikidata.org/entity/Q127861'), ('histCname', 'Khedivate of Egypt')]
[('histC', 'http://www.wikidata.org/entity/Q177819'), ('histCname', 'Old Kingdom of Egypt')]
[('histC', 'http://www.wikidata.org/entity/Q124943'), ('histCname', 'Kingdom of Egypt')]
[('histC', 'http://www.wikidata.org/entity/Q3087763'), ('histCname', 'Republic of Egypt (1953–1958)')]
[('histC', 'http://www.wikidata.org/entity/Q11768'), ('histCname', 'Ancient Egypt')]
[('histC', 'http://www.wikidata.org/entity/Q541455'), ('histCname', 'Anglo-Egyptian Sudan')]
[('histC', 'http://www.wikidata.org/entity/Q180568'), ('histCname', 'New Kingdom of Egypt')]
[('histC', 'http://www.wikidata.org/entity/Q191324'), ('histCname', 'Middle Kingdom of Egypt')]
[('histC', 'http://www.wikidata.org/entity/Q187979'), ('histCname', 'Early Dynastic Period of Egypt')]
9


In [9]:
#SECOND INVESTIGATION: Let's take use this previous query result :
#[('p', 'http://www.wikidata.org/prop/direct/P88'), ('Pname', 'commissioned by'), ('obj', 'http://www.wikidata.org/entity/Q189371'), ('Oname', 'Sneferu')]

queryString = """
SELECT DISTINCT ?obj ?Oname ?p ?Pname
WHERE {
    wd:Q189371 ?p ?obj. #Sneferu(pharaoh)
    
   ?obj sc:name ?Oname.
   ?p sc:name ?Pname.
}

LIMIT 50
"""

print("Results")
x=run_query(queryString)

# We see Ancient Egypt linked to "country of citizenship" property

Results
[('obj', 'http://www.wikidata.org/entity/Q372436'), ('Oname', 'statesperson'), ('p', 'http://www.wikidata.org/prop/direct/P106'), ('Pname', 'occupation')]
[('obj', 'http://www.wikidata.org/entity/Q135332'), ('Oname', 'Red Pyramid'), ('p', 'http://www.wikidata.org/prop/direct/P119'), ('Pname', 'place of burial')]
[('obj', 'http://www.wikidata.org/entity/Q602358'), ('Oname', 'Brockhaus and Efron Encyclopedic Dictionary'), ('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source')]
[('obj', 'http://www.wikidata.org/entity/Q46002746'), ('Oname', 'Dictionary of African Biography'), ('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source')]
[('obj', 'http://www.wikidata.org/entity/Q79'), ('Oname', 'Egypt'), ('p', 'http://www.wikidata.org/prop/direct/P19'), ('Pname', 'place of birth')]
[('obj', 'http://www.wikidata.org/entity/Q79'), ('Oname', 'Egypt'), ('p', 'http://www.wikidata.org/prop/direct/P20'), ('Pname', 'place of death')]
[(

In [10]:
#Let's check the type of the node related to the "commissioned by" property

queryString = """
SELECT DISTINCT ?comissioner ?Cname
WHERE {
   wd:Q189371 wdt:P31 ?comissioner. #Sneferu a

   ?comissioner sc:name ?Cname.
}

LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('comissioner', 'http://www.wikidata.org/entity/Q5'), ('Cname', 'human')]
1


In [11]:
#We can finally build the final query, knowing the comissioners are human and knowing their country of citizenship :

queryString = """
SELECT DISTINCT ?country ?Cname
WHERE {
   wd:Q13444093 ?p ?obj . #MeidumPyramid
   ?obj wdt:P31 wd:Q5; #a human
        wdt:P27 ?country. #countryofcitizenship
        
   ?country sc:name ?Cname.
}
GROUP BY ?p
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q11768'), ('Cname', 'Ancient Egypt')]
1


Final query for this task

In [12]:
# Final query

queryString = """
SELECT DISTINCT ?country ?Cname
WHERE {
   wd:Q13444093 ?p ?obj . #MeidumPyramid
   ?obj wdt:P31 wd:Q5; #a human
        wdt:P27 ?country. #countryofcitizenship
        
   ?country sc:name ?Cname.
}
GROUP BY ?p
LIMIT 50
"""

print("Results")
x=run_query(queryString)

#BGP found: [('histC', 'http://www.wikidata.org/entity/Q11768'), ('histCname', 'Ancient Egypt')]

Results
[('country', 'http://www.wikidata.org/entity/Q11768'), ('Cname', 'Ancient Egypt')]
1


## Task 2
Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

In [6]:
#Thanks to this previous query, we got the BGP of pharaos which intuitively should be useful

queryString = """
SELECT DISTINCT ?obj ?Oname ?p ?Pname
WHERE {
    wd:Q189371 ?p ?obj. #Sneferu (pharaoh)
    
   ?obj sc:name ?Oname.
   ?p sc:name ?Pname.
}

LIMIT 50
"""

print("Results")
x=run_query(queryString)

# [('obj', 'http://www.wikidata.org/entity/Q37110'), ('Oname', 'pharaoh'), ('p', 'http://www.wikidata.org/prop/direct/P39'), ('Pname', 'position held')]

Results
[('obj', 'http://www.wikidata.org/entity/Q372436'), ('Oname', 'statesperson'), ('p', 'http://www.wikidata.org/prop/direct/P106'), ('Pname', 'occupation')]
[('obj', 'http://www.wikidata.org/entity/Q135332'), ('Oname', 'Red Pyramid'), ('p', 'http://www.wikidata.org/prop/direct/P119'), ('Pname', 'place of burial')]
[('obj', 'http://www.wikidata.org/entity/Q602358'), ('Oname', 'Brockhaus and Efron Encyclopedic Dictionary'), ('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source')]
[('obj', 'http://www.wikidata.org/entity/Q46002746'), ('Oname', 'Dictionary of African Biography'), ('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source')]
[('obj', 'http://www.wikidata.org/entity/Q79'), ('Oname', 'Egypt'), ('p', 'http://www.wikidata.org/prop/direct/P19'), ('Pname', 'place of birth')]
[('obj', 'http://www.wikidata.org/entity/Q79'), ('Oname', 'Egypt'), ('p', 'http://www.wikidata.org/prop/direct/P20'), ('Pname', 'place of death')]
[(

In [12]:
#We also have seen previously that Comissioners are Human, so pharaohs are also human. 
#Let's look at human properties to see if there is some related to date of birth and/or family

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname
WHERE {
   wd:Q5 ?p ?obj. #Human

   ?p sc:name ?Pname.
   ?obj sc:name ?Oname.
   FILTER(REGEX(?Oname,"(birth|born|child|kid|parent|family)"))

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

#We get here the date of birth :('obj', 'http://www.wikidata.org/entity/P569'), ('Oname', 'date of birth')]
#And the child property : ('obj', 'http://www.wikidata.org/entity/P40'), ('Oname', 'child')]

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('Pname', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q17534637'), ('Oname', 'Template:Infobox person/Wikidata')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P40'), ('Oname', 'child')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P19'), ('Oname', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1636'), ('Oname', 'date of baptism in early childhood')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata.org/entity/P1477'), ('Oname', 'birth name')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('Pname', 'properties for this type'), ('obj', 'http://www.wikidata

In [13]:
#Let's check at the date of birth format of our known pharaoh (comissioner)

queryString = """
SELECT ?bdate
WHERE {
   wd:Q189371 wdt:P569 ?bdate. #Sneferu dateofbirth
}

"""

print("Results")
x=run_query(queryString)

Results
[('bdate', '-2700-01-01T00:00:00Z')]
1


In [13]:
#Let's try the Filter function on this date format

queryString = """
SELECT ?s ?human ?bdate
WHERE {
   ?s wdt:P31 wd:Q5; #a Human
      wdt:P569 ?bdate. #dateofbirth
      
      ?s sc:name ?human.
      FILTER(?bdate < "1000-01-01T00:00:00Z")
}
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q5664519'), ('human', 'Aldonza Martínez de Silva'), ('bdate', 'nodeID://b325171')]
[('s', 'http://www.wikidata.org/entity/Q4285745'), ('human', 'Mahmud Tarabi'), ('bdate', 'nodeID://b272909')]
[('s', 'http://www.wikidata.org/entity/Q11265554'), ('human', 'I Noi wang'), ('bdate', 'nodeID://b326627')]
[('s', 'http://www.wikidata.org/entity/Q2881314'), ('human', 'Echo'), ('bdate', 'nodeID://b324033')]
[('s', 'http://www.wikidata.org/entity/Q6378167'), ('human', 'Katsunuma Nobutomo'), ('bdate', 'nodeID://b325431')]
5


In [14]:
#birthdates are not printing and filters are wrong (manual verification), let's convert it to xsd:dateTime

queryString = """
SELECT ?s ?human ?bdate
WHERE {
   ?s wdt:P31 wd:Q5; #a Human
      wdt:P569 ?bdate; #dateofbirth
      
      sc:name ?human.
      FILTER(?bdate < "1000-01-01T00:00:00Z"^^xsd:dateTime)
}
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1280296'), ('human', 'Deng Yu'), ('bdate', '0002-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q16078406'), ('human', 'Deng Xun'), ('bdate', '0050-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q167480'), ('human', 'Empress Deng Sui'), ('bdate', '0081-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q12506564'), ('human', 'Pseudo-Apollodorus'), ('bdate', '0050-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q193800'), ('human', 'Juvenal'), ('bdate', '0055-01-01T00:00:00Z')]
5


In [15]:
#We now know how to handle birth dates, let's check now if the pharaoh comissioner has some child with the property we found

queryString = """
SELECT ?child ?Cname
WHERE {
   wd:Q189371 wdt:P40 ?child. #Sneferu hasChild
   ?child sc:name ?Cname
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('child', 'http://www.wikidata.org/entity/Q122426'), ('Cname', 'Henutsen')]
[('child', 'http://www.wikidata.org/entity/Q161904'), ('Cname', 'Khufu')]
[('child', 'http://www.wikidata.org/entity/Q387014'), ('Cname', 'Ankhhaf')]
[('child', 'http://www.wikidata.org/entity/Q453219'), ('Cname', 'Nefertkau I')]
[('child', 'http://www.wikidata.org/entity/Q464954'), ('Cname', 'Meritites I')]
[('child', 'http://www.wikidata.org/entity/Q471584'), ('Cname', 'Nefermaat')]
[('child', 'http://www.wikidata.org/entity/Q7244639'), ('Cname', 'Princess Hetepheres')]
[('child', 'http://www.wikidata.org/entity/Q746710'), ('Cname', 'Prince Rahotep')]
[('child', 'http://www.wikidata.org/entity/Q760610'), ('Cname', 'Ranefer')]
[('child', 'http://www.wikidata.org/entity/Q760621'), ('Cname', 'Kanefer')]
[('child', 'http://www.wikidata.org/entity/Q6987026'), ('Cname', 'Nefertnesu')]
[('child', 'http://www.wikidata.org/entity/Q13638694'), ('Cname', 'Iynefer I')]
[('child', 'http://www.wikidata.org/entity/

In [16]:
#Try #1 with all the collected above info: in order: ?s a pharaoh, ?s from Ancient Egypt, ?s has ?children, ?s has ?bdate
#result should be Pharao URI / Pharao name / #Children but i keep bdate for now to check results

queryString = """
SELECT DISTINCT ?pharaoh ?Pname ?bdate count(?children) as ?count
WHERE {
  ?pharaoh wdt:P39 wd:Q37110; #PositionHeld Pharaoh
           wdt:P27 wd:Q11768; #Nationality AncientEgypt
           wdt:P40 ?children; #hasChild
           wdt:P569 ?bdate; #dateofbirth
           
           sc:name ?Pname.
           
    FILTER(?bdate > "-1500-01-01T00:00:00Z"^^xsd:dateTime && ?bdate < "-1000-01-01T00:00:00Z"^^xsd:dateTime).
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

#Dates ok

Results
[('pharaoh', 'http://www.wikidata.org/entity/Q4009'), ('Pname', 'Ramesses IX'), ('bdate', '-1150-01-01T00:00:00Z'), ('count', '2')]
[('pharaoh', 'http://www.wikidata.org/entity/Q158043'), ('Pname', 'Merneptah'), ('bdate', '-1300-01-01T00:00:00Z'), ('count', '4')]
[('pharaoh', 'http://www.wikidata.org/entity/Q444965'), ('Pname', 'Pinedjem I'), ('bdate', '-1200-01-01T00:00:00Z'), ('count', '8')]
[('pharaoh', 'http://www.wikidata.org/entity/Q42606'), ('Pname', 'Amenhotep III'), ('bdate', '-1403-01-01T00:00:00Z'), ('count', '10')]
[('pharaoh', 'http://www.wikidata.org/entity/Q313121'), ('Pname', 'Ramesses XI'), ('bdate', '-1200-01-01T00:00:00Z'), ('count', '3')]
[('pharaoh', 'http://www.wikidata.org/entity/Q705367'), ('Pname', 'Menkheperre'), ('bdate', '-1100-01-01T00:00:00Z'), ('count', '6')]
[('pharaoh', 'http://www.wikidata.org/entity/Q153344'), ('Pname', 'Meritaten'), ('bdate', '-1400-01-01T00:00:00Z'), ('count', '1')]
[('pharaoh', 'http://www.wikidata.org/entity/Q40930'), ('Pn

In [17]:
#Query working but not required result format 

queryString = """
SELECT ?pharaoh 
WHERE {
  ?pharaoh wdt:P39 wd:Q37110; #PositionHeld Pharaoh
           wdt:P27 wd:Q11768; #Nationality AncientEgypt
           wdt:P40 ?children; #hasChild
           wdt:P569 ?bdate; #dateofbirth
           
           sc:name ?Pname.   
           FILTER(?bdate > "-1500-01-01T00:00:00Z"^^xsd:dateTime && ?bdate < "-1000-01-01T00:00:00Z"^^xsd:dateTime).

           }
GROUP BY ?pharaoh 
HAVING (count(?children) > 8)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('pharaoh', 'http://www.wikidata.org/entity/Q81794')]
[('pharaoh', 'http://www.wikidata.org/entity/Q42606')]
[('pharaoh', 'http://www.wikidata.org/entity/Q1528')]
[('pharaoh', 'http://www.wikidata.org/entity/Q152230')]
[('pharaoh', 'http://www.wikidata.org/entity/Q1523')]
5


In [18]:
queryString = """
SELECT DISTINCT ?pharaoh ?Pname (count(?pharaoh) as ?count)
WHERE {
  ?pharaoh wdt:P39 wd:Q37110; #PositionHeld Pharaoh
           wdt:P27 wd:Q11768; #Nationality AncientEgypt
           wdt:P40 ?children; #hasChild
           wdt:P569 ?bdate; #dateofbirth
           
           sc:name ?Pname.
           
    FILTER(?bdate > "-1500-01-01T00:00:00Z"^^xsd:dateTime && ?bdate < "-1000-01-01T00:00:00Z"^^xsd:dateTime).
}
GROUP BY ?pharaoh ?Pname
HAVING(COUNT(?children) > 8)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('pharaoh', 'http://www.wikidata.org/entity/Q1528'), ('Pname', 'Ramesses III'), ('count', '10')]
[('pharaoh', 'http://www.wikidata.org/entity/Q1523'), ('Pname', 'Ramesses II'), ('count', '18')]
[('pharaoh', 'http://www.wikidata.org/entity/Q42606'), ('Pname', 'Amenhotep III'), ('count', '10')]
[('pharaoh', 'http://www.wikidata.org/entity/Q152230'), ('Pname', 'Tiye'), ('count', '9')]
[('pharaoh', 'http://www.wikidata.org/entity/Q81794'), ('Pname', 'Akhenaten'), ('count', '9')]
5


Final query for this task

In [19]:
# Final query 

queryString = """
SELECT DISTINCT ?pharaoh ?Pname (count(?pharaoh) as ?count)
WHERE {
  ?pharaoh wdt:P39 wd:Q37110; #PositionHeld Pharaoh
           wdt:P27 wd:Q11768; #Nationality AncientEgypt
           wdt:P40 ?children; #hasChild
           wdt:P569 ?bdate; #dateofbirth
           
           sc:name ?Pname.
           
    FILTER(?bdate > "-1500-01-01T00:00:00Z"^^xsd:dateTime && ?bdate < "-1000-01-01T00:00:00Z"^^xsd:dateTime).
}
GROUP BY ?pharaoh ?Pname
HAVING(COUNT(?children) > 8)
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('pharaoh', 'http://www.wikidata.org/entity/Q1528'), ('Pname', 'Ramesses III'), ('count', '10')]
[('pharaoh', 'http://www.wikidata.org/entity/Q1523'), ('Pname', 'Ramesses II'), ('count', '18')]
[('pharaoh', 'http://www.wikidata.org/entity/Q42606'), ('Pname', 'Amenhotep III'), ('count', '10')]
[('pharaoh', 'http://www.wikidata.org/entity/Q152230'), ('Pname', 'Tiye'), ('count', '9')]
[('pharaoh', 'http://www.wikidata.org/entity/Q81794'), ('Pname', 'Akhenaten'), ('count', '9')]
5


## Task 3
Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

In [20]:
#Let's check what Ancient Egypt is instance of (thanks to the BGP found in task 1)

queryString = """
SELECT DISTINCT ?Period ?PeriodType
WHERE {
   wd:Q11768 wdt:P31 ?Period. #AncientEgypt a

   ?Period sc:name ?PeriodType.
}
"""

print("Results")
x=run_query(queryString)

Results
[('Period', 'http://www.wikidata.org/entity/Q28171280'), ('PeriodType', 'ancient civilization')]
[('Period', 'http://www.wikidata.org/entity/Q3024240'), ('PeriodType', 'historical country')]
[('Period', 'http://www.wikidata.org/entity/Q3502482'), ('PeriodType', 'cultural region')]
[('Period', 'http://www.wikidata.org/entity/Q1620908'), ('PeriodType', 'historical region')]
4


In [21]:
#We obtained the ancient civilization URI, we can get its instances :

queryString = """
SELECT DISTINCT ?s ?CountryName
WHERE {
   ?s wdt:P31 wd:Q28171280 . #a ancientCivilization
   
   ?s sc:name ?CountryName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q35355'), ('CountryName', 'Sumer')]
[('s', 'http://www.wikidata.org/entity/Q25558247'), ('CountryName', 'Chinese civilization')]
[('s', 'http://www.wikidata.org/entity/Q38060'), ('CountryName', 'Gaul')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('CountryName', 'Minoan civilization')]
[('s', 'http://www.wikidata.org/entity/Q173082'), ('CountryName', 'Dacia')]
[('s', 'http://www.wikidata.org/entity/Q2429397'), ('CountryName', 'Ancient Carthage')]
[('s', 'http://www.wikidata.org/entity/Q3327860'), ('CountryName', 'Celtic kingdom of Ireland')]
[('s', 'http://www.wikidata.org/entity/Q193748'), ('CountryName', 'history of ancient Israel and Judah')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('CountryName', 'Ancient Egypt')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('CountryName', 'Ancient Greece')]
[('s', 'http://www.wikidata.org/entity/Q41741'), ('CountryName', 'Thrace')]
[('s', 'http://www.wikidata.org/entity/Q128904'), 

In [22]:
#We are required to have their duration, let's if we have them (lloking at related obj/prop)

queryString = """
SELECT DISTINCT ?obj ?Oname
WHERE {
   wd:Q28171280 ?p ?obj. #AncientCivilization

   ?p sc:name ?Pname.
   ?obj sc:name ?Oname.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q8432'), ('Oname', 'civilization')]
1


In [23]:
#Same as above but for the "civilization" object (related obj)

queryString = """
SELECT DISTINCT ?obj ?Oname
WHERE {
   wd:Q8432 ?p ?obj. #Civilization

   ?obj sc:name ?Oname.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q8425'), ('Oname', 'society')]
[('obj', 'http://www.wikidata.org/entity/Q2041543'), ('Oname', 'Ottův slovník naučný')]
[('obj', 'http://www.wikidata.org/entity/Q5460604'), ('Oname', 'Wikipedia:Vital articles')]
[('obj', 'http://www.wikidata.org/entity/Q1868663'), ('Oname', 'Civilization')]
[('obj', 'http://www.wikidata.org/entity/Q1156970'), ('Oname', 'humanity')]
[('obj', 'http://www.wikidata.org/entity/Q83781188'), ('Oname', 'primitive culture')]
[('obj', 'http://www.wikidata.org/entity/Q6576454'), ('Oname', 'Category:Civilizations')]
[('obj', 'http://www.wikidata.org/entity/Q8036557'), ('Oname', 'world community')]
8


In [24]:
#Let's do the same check check for historical country

queryString = """
SELECT DISTINCT ?obj ?Oname
WHERE {
   wd:Q3024240 ?p ?obj. #HistoricalCountry

   ?obj sc:name ?Oname.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q28864179'), ('Oname', 'proposed country')]
[('obj', 'http://www.wikidata.org/entity/Q19953632'), ('Oname', 'former administrative territorial entity')]
[('obj', 'http://www.wikidata.org/entity/P580'), ('Oname', 'start time')]
[('obj', 'http://www.wikidata.org/entity/P582'), ('Oname', 'end time')]
[('obj', 'http://www.wikidata.org/entity/P576'), ('Oname', 'dissolved, abolished or demolished date')]
[('obj', 'http://www.wikidata.org/entity/P17'), ('Oname', 'country')]
[('obj', 'http://www.wikidata.org/entity/P571'), ('Oname', 'inception')]
[('obj', 'http://www.wikidata.org/entity/P36'), ('Oname', 'capital')]
[('obj', 'http://www.wikidata.org/entity/P1365'), ('Oname', 'replaces')]
[('obj', 'http://www.wikidata.org/entity/P1366'), ('Oname', 'replaced by')]
[('obj', 'http://www.wikidata.org/entity/P3842'), ('Oname', 'located in present-day administrative territorial entity')]
[('obj', 'http://www.wikidata.org/entity/Q6036853'), ('Oname', 'Te

In [25]:
#We have start time and endtime, let's check the format for the individual AncientEgypt, and how we can get the duration

queryString = """
SELECT DISTINCT ?start ?end (year(?end)-year(?start) as ?duration)
WHERE {
   wd:Q11768 wdt:P580 ?start; #AncientEgypt startTime
             wdt:P582 ?end.
             
}
"""

print("Results")
x=run_query(queryString)

Results
[('start', '-4000-01-01T00:00:00Z'), ('end', '0395-01-01T00:00:00Z'), ('duration', '4395')]
1


In [26]:
#Query with the optional applied on start and end :

queryString = """
SELECT DISTINCT ?s ?countryName ?start ?end 
WHERE {
    ?s wdt:P31 wd:Q28171280. #AncientCivilization
    OPTIONAL { ?s wdt:P580 ?start }. #hasStart
    OPTIONAL { ?s wdt:P582 ?end }. #hasEnd
       
    ?s sc:name ?countryName.
}
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('countryName', 'Minoan civilization'), ('start', '-3299-01-01T00:00:00Z'), ('end', '-1099-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('countryName', 'Minoan civilization'), ('start', '-2000-01-01T00:00:00Z'), ('end', '-1099-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('countryName', 'Ancient Egypt'), ('start', '-4000-01-01T00:00:00Z'), ('end', '0395-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('countryName', 'Ancient Greece'), ('start', '-1200-01-01T00:00:00Z'), ('end', '0600-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('countryName', 'Indus Valley Civilization'), ('start', '-2999-01-01T00:00:00Z'), ('end', '-1799-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q1551054'), ('countryName', 'Israelite period'), ('start', '-1199-01-01T00:00:00Z'), ('end', '-585-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q33278

In [27]:
#For the final query, we need to check if start and end or bound, otherwise it throws an error (because we would be calculating duration on null values)
#To do so, we need an if statement if(bool, do if bool=True, do if bool=False)

queryString = """
SELECT DISTINCT ?s ?countryName (IF(BOUND(?end) && BOUND(?start), year(?end)-year(?start), ?null) as ?duration)
WHERE {
    ?s wdt:P31 wd:Q28171280. #AncientCivilization
    OPTIONAL { ?s wdt:P580 ?start }. #hasStart
    OPTIONAL { ?s wdt:P582 ?end }. #hasEnd
       
    ?s sc:name ?countryName.
}
"""

print("Results")
x=run_query(queryString)
#Note that Minoan civilization has 2 start dates in wikidata (is it an error?), so 2 rows are returned here

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('countryName', 'Minoan civilization'), ('duration', '2200')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('countryName', 'Minoan civilization'), ('duration', '901')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('countryName', 'Ancient Egypt'), ('duration', '4395')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('countryName', 'Ancient Greece'), ('duration', '1800')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('countryName', 'Indus Valley Civilization'), ('duration', '1200')]
[('s', 'http://www.wikidata.org/entity/Q1551054'), ('countryName', 'Israelite period'), ('duration', '614')]
[('s', 'http://www.wikidata.org/entity/Q3327860'), ('countryName', 'Celtic kingdom of Ireland')]
[('s', 'http://www.wikidata.org/entity/Q35355'), ('countryName', 'Sumer')]
[('s', 'http://www.wikidata.org/entity/Q25558247'), ('countryName', 'Chinese civilization')]
[('s', 'http://www.wikidata.org/entity/Q38060'), ('countryName', 'G

Final query for this task

In [28]:
# Final query

queryString = """
SELECT DISTINCT ?s ?countryName (IF(BOUND(?end) && BOUND(?start), year(?end)-year(?start), ?null) as ?duration)
WHERE {
    ?s wdt:P31 wd:Q28171280. #AncientCivilization
    OPTIONAL { ?s wdt:P580 ?start }. #hasStart
    OPTIONAL { ?s wdt:P582 ?end }. #hasEnd
       
    ?s sc:name ?countryName.
}
"""

print("Results")
x=run_query(queryString)

#Note that Minoan civilization has 2 start dates in wikidata (is it an error?), so 2 rows are returned here

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('countryName', 'Minoan civilization'), ('duration', '2200')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('countryName', 'Minoan civilization'), ('duration', '901')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('countryName', 'Ancient Egypt'), ('duration', '4395')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('countryName', 'Ancient Greece'), ('duration', '1800')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('countryName', 'Indus Valley Civilization'), ('duration', '1200')]
[('s', 'http://www.wikidata.org/entity/Q1551054'), ('countryName', 'Israelite period'), ('duration', '614')]
[('s', 'http://www.wikidata.org/entity/Q3327860'), ('countryName', 'Celtic kingdom of Ireland')]
[('s', 'http://www.wikidata.org/entity/Q35355'), ('countryName', 'Sumer')]
[('s', 'http://www.wikidata.org/entity/Q25558247'), ('countryName', 'Chinese civilization')]
[('s', 'http://www.wikidata.org/entity/Q38060'), ('countryName', 'G

## Task 4
Identify the BGP for archeologist.

In [3]:
#In a query of task 1, we managed to find the archaeological site BGP : [('Otype', 'http://www.wikidata.org/entity/Q839954'), ('Oname', 'archaeological site')
#Let's check the related properties

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname
WHERE {
   wd:Q839954 ?p ?obj. #ArchaeologicalSite

   ?p sc:name ?Pname.
   ?obj sc:name ?Oname.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q2657718'), ('Oname', 'Armenian Soviet Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('Pname', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q210272'), ('Oname', 'cultural heritage')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q103892210'), ('Oname', 'Green Map Icons')]
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('Pname', 'studied by'), ('obj', 'http://www.wikidata.org/entity/Q23498'), ('Oname', 'archaeology')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('Pname', 'Wikidata property'), ('obj', 'http://www.wikidata.org/entity/P9047'), ('Oname', 'archaeological site of')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('Pname', "topic's main template"), ('obj', 'http://www.wikidata.org/entity/Q11705134'), ('Oname', 'Te

In [39]:
#We see the property studied by bound to archaeology, let's see their properties/objects

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname
WHERE {
   wd:Q23498 ?p ?obj. #Archaeology

   ?p sc:name ?Pname.
   ?obj sc:name ?Oname.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

#We end up finding the BGP for archaeologist, bound to the property practiced by : ('obj', 'http://www.wikidata.org/entity/Q3621491'), ('Oname', 'archaeologist'))

Results
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('Pname', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q1066186'), ('Oname', 'study of history')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('Pname', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q11862829'), ('Oname', 'academic discipline')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q19180675'), ('Oname', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q19211082'), ('Oname', 'Orthodox Theological Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source'), ('obj', 'http://www.wikidata.org/e

Final query for this task

In [3]:
# Final query

queryString = """
SELECT DISTINCT ?obj ?Oname
WHERE {
   wd:Q23498 ?p ?obj. #Archaeology
   
   ?obj sc:name ?Oname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

# [('obj', 'http://www.wikidata.org/entity/Q3621491'), ('Oname', 'archaeologist')]

Results
[('obj', 'http://www.wikidata.org/entity/Q1066186'), ('Oname', 'study of history')]
[('obj', 'http://www.wikidata.org/entity/Q11862829'), ('Oname', 'academic discipline')]
[('obj', 'http://www.wikidata.org/entity/Q19180675'), ('Oname', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('obj', 'http://www.wikidata.org/entity/Q19211082'), ('Oname', 'Orthodox Theological Encyclopedia')]
[('obj', 'http://www.wikidata.org/entity/Q2657718'), ('Oname', 'Armenian Soviet Encyclopedia')]
[('obj', 'http://www.wikidata.org/entity/Q3181656'), ('Oname', 'The Nuttall Encyclopædia')]
[('obj', 'http://www.wikidata.org/entity/Q34749'), ('Oname', 'social science')]
[('obj', 'http://www.wikidata.org/entity/Q4114391'), ('Oname', 'Sytin Military Encyclopedia')]
[('obj', 'http://www.wikidata.org/entity/Q4173137'), ('Oname', 'Jewish Encyclopedia of Brockhaus and Efron')]
[('obj', 'http://www.wikidata.org/entity/Q4671286'), ('Oname', 'academic major')]
[('obj', 'http://www.wikidata.org/entity/Q602

## Task 5
For each ancient civilization find the number of archeologists who worked on it (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

In [5]:
#Let's get back to our list of ancient civilizations

queryString = """
SELECT DISTINCT ?civ ?CivName
WHERE {
   ?civ wdt:P31 wd:Q28171280 . #a ancientCivilization
   
   ?civ sc:name ?CivName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('civ', 'http://www.wikidata.org/entity/Q35355'), ('CivName', 'Sumer')]
[('civ', 'http://www.wikidata.org/entity/Q25558247'), ('CivName', 'Chinese civilization')]
[('civ', 'http://www.wikidata.org/entity/Q38060'), ('CivName', 'Gaul')]
[('civ', 'http://www.wikidata.org/entity/Q134178'), ('CivName', 'Minoan civilization')]
[('civ', 'http://www.wikidata.org/entity/Q173082'), ('CivName', 'Dacia')]
[('civ', 'http://www.wikidata.org/entity/Q2429397'), ('CivName', 'Ancient Carthage')]
[('civ', 'http://www.wikidata.org/entity/Q3327860'), ('CivName', 'Celtic kingdom of Ireland')]
[('civ', 'http://www.wikidata.org/entity/Q193748'), ('CivName', 'history of ancient Israel and Judah')]
[('civ', 'http://www.wikidata.org/entity/Q11768'), ('CivName', 'Ancient Egypt')]
[('civ', 'http://www.wikidata.org/entity/Q11772'), ('CivName', 'Ancient Greece')]
[('civ', 'http://www.wikidata.org/entity/Q41741'), ('CivName', 'Thrace')]
[('civ', 'http://www.wikidata.org/entity/Q128904'), ('CivName', 'Elam')]

In [17]:
#Let's check if the nationality corresponds to the ancient civilization name, taking Sneferu pharaoh

queryString = """
SELECT DISTINCT ?natio ?Nname
WHERE { 
   wd:Q189371 wdt:P27 ?natio. #Sneferu nationality
   
   ?natio sc:name ?Nname.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('natio', 'http://www.wikidata.org/entity/Q11768'), ('Nname', 'Ancient Egypt')]
1


In [43]:
#The property wdt:P106 is given in the beginning of the workflow, let's use it by assuming Ancient Egypt has at least one archaeologist and get their name/number :

queryString = """
SELECT DISTINCT ?arch ?Aname
WHERE { 
   ?arch wdt:P31 wd:Q5; #a human
         wdt:P106 wd:Q3621491; #occupation archaeologist
         wdt:P27 wd:Q11768. #natio AncientEgypt
   
   ?arch sc:name ?Aname.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [44]:
#Above query returns empty, let's see if the problem is the "occupation" property

queryString = """
SELECT DISTINCT ?arch ?Aname
WHERE { 
   ?arch wdt:P31 wd:Q5; #a human
         #wdt:P106 wd:Q3621491; #occupation archaeologist
         wdt:P27 wd:Q11768. #natio AncientEgypt
   
   ?arch sc:name ?Aname.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('arch', 'http://www.wikidata.org/entity/Q1216110'), ('Aname', 'Nodjmet')]
[('arch', 'http://www.wikidata.org/entity/Q151828'), ('Aname', 'Djet')]
[('arch', 'http://www.wikidata.org/entity/Q152375'), ('Aname', 'Djer')]
[('arch', 'http://www.wikidata.org/entity/Q19248'), ('Aname', 'Senusret III')]
[('arch', 'http://www.wikidata.org/entity/Q263218'), ('Aname', 'Djedkhonsuefankh')]
[('arch', 'http://www.wikidata.org/entity/Q269265'), ('Aname', 'Pepi I Meryre')]
[('arch', 'http://www.wikidata.org/entity/Q320908'), ('Aname', 'Merenre Nemtyemsaf II')]
[('arch', 'http://www.wikidata.org/entity/Q336395'), ('Aname', 'Smendes')]
[('arch', 'http://www.wikidata.org/entity/Q380676'), ('Aname', 'Psusennes I')]
[('arch', 'http://www.wikidata.org/entity/Q428633'), ('Aname', 'Piankh')]
10


In [49]:
queryString = """
SELECT DISTINCT ?arch ?Aname
WHERE { 
   ?arch wdt:P31 wd:Q5; #a human
         wdt:P106 wd:Q3621491. #occupation archaeologist
   
   ?arch sc:name ?Aname.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

#The problem might then be the nationality 

Results
[('arch', 'http://www.wikidata.org/entity/Q8463726'), ('Aname', 'Chengzuo Shang')]
[('arch', 'http://www.wikidata.org/entity/Q13638340'), ('Aname', 'Dagmar Jelínková')]
[('arch', 'http://www.wikidata.org/entity/Q15466790'), ('Aname', 'Josef Poppelreuter')]
[('arch', 'http://www.wikidata.org/entity/Q15729291'), ('Aname', 'Marino Marini')]
[('arch', 'http://www.wikidata.org/entity/Q15730811'), ('Aname', 'Hartmut Schäfer')]
[('arch', 'http://www.wikidata.org/entity/Q15731701'), ('Aname', 'Pietro Tessieri')]
[('arch', 'http://www.wikidata.org/entity/Q3839667'), ('Aname', 'Luigi Gaetano Marini')]
[('arch', 'http://www.wikidata.org/entity/Q104327'), ('Aname', 'Otto Rossbach')]
[('arch', 'http://www.wikidata.org/entity/Q15906413'), ('Aname', 'Frank Henry Stubbings')]
[('arch', 'http://www.wikidata.org/entity/Q5717733'), ('Aname', 'Henry Balfour')]
10


In [74]:
#I just found out I misread the question, I was looking for archaeologist FROM ancient civilizations, not archaeologist who WORKED ON ancient civilizations
#We need to find a property like "subjectOfStudy" or "fieldOfStudy" that could link an archaeologist and ancient civ 
#Let's explore prop/objects linked to archaeologist

queryString = """
SELECT DISTINCT ?p ?PName
WHERE {
   ?s ?p wd:Q3621491. #archaeologist
   
   ?p sc:name ?PName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

#Maybe main subject property is interesting

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('PName', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('PName', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P180'), ('PName', 'depicts')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('PName', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('PName', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P3095'), ('PName', 'practiced by')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('PName', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('PName', 'is a list of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('PName', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P39'), ('PName', 'position held')]
[('p', 'http://www.wikidata.org/prop/direct/P3989'), ('PName', 'members have occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('PName', 'main subject')]
[('p', 'http://www.wikidata.org/prop

In [80]:
#Let's check the type of Things that have "Ancient Egypt" as field of work

queryString = """
SELECT DISTINCT ?OType ?OName
WHERE {
   ?obj wdt:P101 wd:Q11768. #field AncientEgypt
   ?obj wdt:P31 ?OType. #a
   
   ?OType sc:name ?OName.
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('OType', 'http://www.wikidata.org/entity/Q7315155'), ('OName', 'research center')]
[('OType', 'http://www.wikidata.org/entity/Q5'), ('OName', 'human')]
[('OType', 'http://www.wikidata.org/entity/Q5633421'), ('OName', 'scientific journal')]
3


In [98]:
#Let's start building the query 

queryString = """
SELECT DISTINCT ?arch ?archName ?theme ?themeName
WHERE { 
   ?arch wdt:P31 wd:Q5; #a human 
         wdt:P106 wd:Q3621491; #occupation archaeologist
         wdt:P101 ?theme. #fieldOfWork
    ?theme wdt:P31 wd:Q28171280.  #a AncientCiv
      
   ?arch sc:name ?archName.
   ?theme sc:name ?themeName
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('arch', 'http://www.wikidata.org/entity/Q2342814'), ('archName', 'Alexandre Bertrand'), ('theme', 'http://www.wikidata.org/entity/Q38060'), ('themeName', 'Gaul')]
[('arch', 'http://www.wikidata.org/entity/Q1985476'), ('archName', 'Charles Thomas Newton'), ('theme', 'http://www.wikidata.org/entity/Q11772'), ('themeName', 'Ancient Greece')]
[('arch', 'http://www.wikidata.org/entity/Q1523942'), ('archName', 'Gilbert Charles-Picard'), ('theme', 'http://www.wikidata.org/entity/Q2429397'), ('themeName', 'Ancient Carthage')]
[('arch', 'http://www.wikidata.org/entity/Q64563606'), ('archName', "Vincent Robin d'Arba Desborough"), ('theme', 'http://www.wikidata.org/entity/Q11772'), ('themeName', 'Ancient Greece')]
[('arch', 'http://www.wikidata.org/entity/Q2959998'), ('archName', 'Charles Picard'), ('theme', 'http://www.wikidata.org/entity/Q11772'), ('themeName', 'Ancient Greece')]
[('arch', 'http://www.wikidata.org/entity/Q107185282'), ('archName', 'Orlando Cerasuolo'), ('theme', 'http

In [29]:
#Adding the aggregation

queryString = """
SELECT DISTINCT ?theme ?themeName (count(*) as ?count)
WHERE { 
   ?arch wdt:P31 wd:Q5; #a human 
         wdt:P106 wd:Q3621491; #occupation archaeologist
         wdt:P101 ?theme. #fieldOfWork
    ?theme wdt:P31 wd:Q28171280.  #a AncientCiv
      
   ?arch sc:name ?archName.
   ?theme sc:name ?themeName
}
GROUP BY ?theme ?themeName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('theme', 'http://www.wikidata.org/entity/Q11767'), ('themeName', 'Mesopotamia'), ('count', '1')]
[('theme', 'http://www.wikidata.org/entity/Q193748'), ('themeName', 'history of ancient Israel and Judah'), ('count', '1')]
[('theme', 'http://www.wikidata.org/entity/Q11772'), ('themeName', 'Ancient Greece'), ('count', '7')]
[('theme', 'http://www.wikidata.org/entity/Q1551054'), ('themeName', 'Israelite period'), ('count', '2')]
[('theme', 'http://www.wikidata.org/entity/Q2429397'), ('themeName', 'Ancient Carthage'), ('count', '7')]
[('theme', 'http://www.wikidata.org/entity/Q41741'), ('themeName', 'Thrace'), ('count', '1')]
[('theme', 'http://www.wikidata.org/entity/Q38060'), ('themeName', 'Gaul'), ('count', '3')]
[('theme', 'http://www.wikidata.org/entity/Q28567'), ('themeName', 'Maya civilization'), ('count', '1')]
8


In [112]:
#We also need the civilizations with No Archaeologists :

queryString = """
SELECT DISTINCT ?civ ?CivName
WHERE {
   ?civ wdt:P31 wd:Q28171280. #a ancientCivilization
   
   OPTIONAL { ?arch wdt:P31 wd:Q5 }. #a human 
   OPTIONAL { ?arch wdt:P106 wd:Q3621491 }. #occupation archaeologist
   OPTIONAL { ?arch wdt:P101 ?civ }. #mainSubject
                    
   ?civ sc:name ?CivName.
}
LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
The operation failed timed out


In [30]:
#One optional 

queryString = """
SELECT DISTINCT ?civ ?CivName (count(?arch) as ?count)
WHERE {
   ?civ wdt:P31 wd:Q28171280. #a ancientCivilization
   
   OPTIONAL { ?arch wdt:P31 wd:Q5; 
                    wdt:P106 wd:Q3621491; 
                    wdt:P101 ?civ. } 
                    
   ?civ sc:name ?CivName.
}
GROUP BY ?civ ?CivName
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('civ', 'http://www.wikidata.org/entity/Q11767'), ('CivName', 'Mesopotamia'), ('count', '1')]
[('civ', 'http://www.wikidata.org/entity/Q193748'), ('CivName', 'history of ancient Israel and Judah'), ('count', '1')]
[('civ', 'http://www.wikidata.org/entity/Q11772'), ('CivName', 'Ancient Greece'), ('count', '7')]
[('civ', 'http://www.wikidata.org/entity/Q173082'), ('CivName', 'Dacia'), ('count', '0')]
[('civ', 'http://www.wikidata.org/entity/Q1364601'), ('CivName', 'Gallo-Roman culture'), ('count', '0')]
[('civ', 'http://www.wikidata.org/entity/Q768212'), ('CivName', 'Turco-Mongol'), ('count', '0')]
[('civ', 'http://www.wikidata.org/entity/Q1551054'), ('CivName', 'Israelite period'), ('count', '2')]
[('civ', 'http://www.wikidata.org/entity/Q28405'), ('CivName', 'Gojoseon'), ('count', '0')]
[('civ', 'http://www.wikidata.org/entity/Q2429397'), ('CivName', 'Ancient Carthage'), ('count', '7')]
[('civ', 'http://www.wikidata.org/entity/Q185068'), ('CivName', 'Urartu'), ('count', '0')]


In [31]:
#I think the previous query result is incomplete, let's check if other properties link archaeologists and ancient civilizations :

queryString = """
SELECT DISTINCT ?did ?didName
WHERE {
     ?arch wdt:P31 wd:Q5; #a human 
         wdt:P106 wd:Q3621491; #occupation archaeologist
         ?did ?something.
         
  ?something wdt:P31 wd:Q28171280. #a AncientCiv
  
  ?did sc:name ?didName

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('did', 'http://www.wikidata.org/prop/direct/P101'), ('didName', 'field of work')]
1


In [32]:
#Then the problem may be that archaeologists do work related to ancient civilizations, but not "directly".
#For example "Egyptology" is about Ancient Egypt but I'm not taking it into account in my previous answer attempt.
#Let's explore a property found in a previous result : [('p', 'http://www.wikidata.org/prop/direct/P2578'), ('Pname', 'studies') 
#applied to subclasses of archaeology

queryString = """
SELECT DISTINCT ?field ?fieldName
WHERE {
    ?civ wdt:P31 wd:Q28171280. #AncientCiv
    ?field wdt:P279 wd:Q23498; #subclassOf archaeology
          wdt:P2578 ?civ. #studies
          
   ?field sc:name ?fieldName
  }
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('field', 'http://www.wikidata.org/entity/Q145903'), ('fieldName', 'egyptology')]
1


In [33]:
#See if there are archaeologists related to the field

queryString = """
SELECT DISTINCT ?civ ?civName (count(distinct ?arch) as ?ArchCount)
WHERE {
   ?civ wdt:P31 wd:Q28171280. #a ancientCivilization
   ?field wdt:P279 wd:Q23498; #subclassOf archaeology
          wdt:P2578 ?civ.     #studies
  
     ?arch wdt:P31 wd:Q5;       #a human 
         wdt:P106 wd:Q3621491;  #occupation archaeologist
         wdt:P101 ?field.       #mainSubject

   ?civ sc:name ?civName.
}
GROUP BY ?civ ?civName
LIMIT 20
"""

print("Results")
x=run_query(queryString)

#We know now that there are 38 archaeologists that study Ancient Egypt.

Results
[('civ', 'http://www.wikidata.org/entity/Q11768'), ('civName', 'Ancient Egypt'), ('ArchCount', '38')]
1


In [52]:
#Let's try now to do the union of both results :

queryString = """
SELECT DISTINCT ?civ ?civName (count(?arch) as ?ArchCount)
WHERE 
{
    {
        {
           SELECT DISTINCT ?arch ?civ ?civName WHERE 
           {
                   ?civ wdt:P31 wd:Q28171280. #a ancientCivilization
                   ?field wdt:P279 wd:Q23498; #subclassOf archaeology
                          wdt:P2578 ?civ.     #studies    

                     ?arch wdt:P31 wd:Q5; #a human 
                           wdt:P106 wd:Q3621491; #occupation archaeologist
                           wdt:P101 ?field. #mainSubject

                     ?civ sc:name ?civName.
            }   
        }
        UNION
        {
            SELECT DISTINCT ?arch ?civ ?civName WHERE 
            {
                   ?civ wdt:P31 wd:Q28171280. #a ancientCivilization

                   OPTIONAL { ?arch wdt:P31 wd:Q5; 
                                    wdt:P106 wd:Q3621491; 
                                    wdt:P101 ?civ. }  #A human  #occupation archaeo  #MainSubject

                   ?civ sc:name ?civName.
            }
        }
    }
}
GROUP BY ?civ ?civName
"""

print("Results")
x=run_query(queryString)

Results
[('civ', 'http://www.wikidata.org/entity/Q11767'), ('civName', 'Mesopotamia'), ('ArchCount', '1')]
[('civ', 'http://www.wikidata.org/entity/Q193748'), ('civName', 'history of ancient Israel and Judah'), ('ArchCount', '1')]
[('civ', 'http://www.wikidata.org/entity/Q11772'), ('civName', 'Ancient Greece'), ('ArchCount', '7')]
[('civ', 'http://www.wikidata.org/entity/Q173082'), ('civName', 'Dacia'), ('ArchCount', '0')]
[('civ', 'http://www.wikidata.org/entity/Q1364601'), ('civName', 'Gallo-Roman culture'), ('ArchCount', '0')]
[('civ', 'http://www.wikidata.org/entity/Q768212'), ('civName', 'Turco-Mongol'), ('ArchCount', '0')]
[('civ', 'http://www.wikidata.org/entity/Q1551054'), ('civName', 'Israelite period'), ('ArchCount', '2')]
[('civ', 'http://www.wikidata.org/entity/Q28405'), ('civName', 'Gojoseon'), ('ArchCount', '0')]
[('civ', 'http://www.wikidata.org/entity/Q2429397'), ('civName', 'Ancient Carthage'), ('ArchCount', '7')]
[('civ', 'http://www.wikidata.org/entity/Q185068'), ('c

Final query for this task

In [57]:
#Final query
#Explanation: I only found 2 ways to link ancient civilizations and archaeologists.
#The first one (corresponding to first subquery) is an "indirect link" :
#I look for archaeologists who STUDY A SUBJECT RELATED TO the ancient civilization, like egyptology. 
#The second one (second subquery) is a "direct link": I look for archaeologists whose MAIN SUBJECT IS the ancient civilization.

queryString = """
SELECT DISTINCT ?civ ?civName (count(?arch) as ?ArchCount)
WHERE 
{
    {
        {
           SELECT DISTINCT ?arch ?civ ?civName WHERE 
           {
                   ?civ wdt:P31 wd:Q28171280. #a ancientCivilization
                   ?field wdt:P279 wd:Q23498; #subclassOf archaeology
                          wdt:P2578 ?civ.     #studies    

                     ?arch wdt:P31 wd:Q5; #a human 
                           wdt:P106 wd:Q3621491; #occupation archaeologist
                           wdt:P101 ?field. #mainSubject

                     ?civ sc:name ?civName.
            }   
        }
        UNION
        {
            SELECT DISTINCT ?arch ?civ ?civName WHERE 
            {
                   ?civ wdt:P31 wd:Q28171280. #a ancientCivilization

                   OPTIONAL { ?arch wdt:P31 wd:Q5; 
                                    wdt:P106 wd:Q3621491; 
                                    wdt:P101 ?civ. }  #A human  #occupation archaeo  #MainSubject

                   ?civ sc:name ?civName.
            }
        }
    }
}
GROUP BY ?civ ?civName
ORDER BY DESC (?ArchCount)
"""

print("Results")
x=run_query(queryString)

Results
[('civ', 'http://www.wikidata.org/entity/Q11768'), ('civName', 'Ancient Egypt'), ('ArchCount', '38')]
[('civ', 'http://www.wikidata.org/entity/Q11772'), ('civName', 'Ancient Greece'), ('ArchCount', '7')]
[('civ', 'http://www.wikidata.org/entity/Q2429397'), ('civName', 'Ancient Carthage'), ('ArchCount', '7')]
[('civ', 'http://www.wikidata.org/entity/Q38060'), ('civName', 'Gaul'), ('ArchCount', '3')]
[('civ', 'http://www.wikidata.org/entity/Q1551054'), ('civName', 'Israelite period'), ('ArchCount', '2')]
[('civ', 'http://www.wikidata.org/entity/Q28567'), ('civName', 'Maya civilization'), ('ArchCount', '2')]
[('civ', 'http://www.wikidata.org/entity/Q11767'), ('civName', 'Mesopotamia'), ('ArchCount', '1')]
[('civ', 'http://www.wikidata.org/entity/Q193748'), ('civName', 'history of ancient Israel and Judah'), ('ArchCount', '1')]
[('civ', 'http://www.wikidata.org/entity/Q41741'), ('civName', 'Thrace'), ('ArchCount', '1')]
[('civ', 'http://www.wikidata.org/entity/Q173082'), ('civName'

## Task 6
Find the BGP for deity.

In [58]:
# We are given in the first array the BGP of Poseidon: wd:Q41127`| Poseidon | node 
#Let's see what Poseidon is instance of

queryString = """
SELECT DISTINCT ?type ?typeName
WHERE {
   wd:Q41127 wdt:P31 ?type. #Poseidon a

   ?type sc:name ?typeName.
}
"""

print("Results")
x=run_query(queryString)

Results
[('type', 'http://www.wikidata.org/entity/Q1916821'), ('typeName', 'water deity')]
[('type', 'http://www.wikidata.org/entity/Q22989102'), ('typeName', 'Greek deity')]
2


In [59]:
#Let's check subclasses

queryString = """
SELECT DISTINCT ?subclass ?subclassName
WHERE {
   wd:Q41127 wdt:P31 ?type. #Poseidon a
   ?type wdt:P279 ?subclass. #subclass
   
   ?subclass sc:name ?subclassName
}
"""

print("Results")
x=run_query(queryString)

Results
[('subclass', 'http://www.wikidata.org/entity/Q95074'), ('subclassName', 'fictional character')]
[('subclass', 'http://www.wikidata.org/entity/Q13405593'), ('subclassName', 'nature deity')]
[('subclass', 'http://www.wikidata.org/entity/Q22988604'), ('subclassName', 'mythological Greek character')]
[('subclass', 'http://www.wikidata.org/entity/Q1936706'), ('subclassName', 'water sprite')]
[('subclass', 'http://www.wikidata.org/entity/Q178885'), ('subclassName', 'deity')]
5


Final query for this task

In [60]:
#Final query

queryString = """
SELECT DISTINCT ?subclass ?subclassName
WHERE {
   wd:Q41127 wdt:P31 ?type. #Poseidon a
   ?type wdt:P279 ?subclass. #subclass
   
   ?subclass sc:name ?subclassName
}
"""

print("Results")
x=run_query(queryString)

#BGP : [('subclass', 'http://www.wikidata.org/entity/Q178885'), ('subclassName', 'deity')]

Results
[('subclass', 'http://www.wikidata.org/entity/Q95074'), ('subclassName', 'fictional character')]
[('subclass', 'http://www.wikidata.org/entity/Q13405593'), ('subclassName', 'nature deity')]
[('subclass', 'http://www.wikidata.org/entity/Q22988604'), ('subclassName', 'mythological Greek character')]
[('subclass', 'http://www.wikidata.org/entity/Q1936706'), ('subclassName', 'water sprite')]
[('subclass', 'http://www.wikidata.org/entity/Q178885'), ('subclassName', 'deity')]
5


## Task 7
List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

In [38]:
#In a previous query we got the following bgp: [('type', 'http://www.wikidata.org/entity/Q22989102'), ('typeName', 'Greek deity')]
#Let's check the instances of Greek gods

queryString = """
SELECT DISTINCT ?s ?SName
WHERE {
   ?s wdt:P31 wd:Q22989102. #Poseidon a
   
   ?s sc:name ?SName
}
LIMIT 20
"""


print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q5081127'), ('SName', 'Acmon')]
[('s', 'http://www.wikidata.org/entity/Q366533'), ('SName', 'Adranus')]
[('s', 'http://www.wikidata.org/entity/Q3362052'), ('SName', 'Pandora')]
[('s', 'http://www.wikidata.org/entity/Q6612'), ('SName', 'Charon')]
[('s', 'http://www.wikidata.org/entity/Q2144233'), ('SName', 'Pheme')]
[('s', 'http://www.wikidata.org/entity/Q1120723'), ('SName', 'Pheme')]
[('s', 'http://www.wikidata.org/entity/Q1862189'), ('SName', 'Paean')]
[('s', 'http://www.wikidata.org/entity/Q1775444'), ('SName', 'Phosphorus')]
[('s', 'http://www.wikidata.org/entity/Q178417'), ('SName', 'Eos')]
[('s', 'http://www.wikidata.org/entity/Q59697'), ('SName', 'Hesperus')]
[('s', 'http://www.wikidata.org/entity/Q1084868'), ('SName', 'Lyssa')]
[('s', 'http://www.wikidata.org/entity/Q1294659'), ('SName', 'Telesphorus')]
[('s', 'http://www.wikidata.org/entity/Q18423351'), ('SName', 'Basileia')]
[('s', 'http://www.wikidata.org/entity/Q185670'), ('SNa

In [6]:
#Let's now check if olympian deity is a subclass of greek deity

queryString = """
SELECT DISTINCT ?s ?SName
WHERE {
   ?s wdt:P279 wd:Q22989102. #subclassOf GreekDeity
   
   ?s sc:name ?SName
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1134605'), ('SName', 'Telchines')]
[('s', 'http://www.wikidata.org/entity/Q247617'), ('SName', 'Curetes')]
[('s', 'http://www.wikidata.org/entity/Q3346693'), ('SName', 'Greek nymph')]
[('s', 'http://www.wikidata.org/entity/Q45713'), ('SName', 'titan')]
[('s', 'http://www.wikidata.org/entity/Q1157625'), ('SName', 'daemon')]
[('s', 'http://www.wikidata.org/entity/Q201411'), ('SName', 'Horae')]
[('s', 'http://www.wikidata.org/entity/Q101609'), ('SName', 'Twelve Olympians')]
[('s', 'http://www.wikidata.org/entity/Q184353'), ('SName', 'Charites')]
[('s', 'http://www.wikidata.org/entity/Q878099'), ('SName', 'Greek primordial deity')]
[('s', 'http://www.wikidata.org/entity/Q66016'), ('SName', 'Muses')]
[('s', 'http://www.wikidata.org/entity/Q180287'), ('SName', 'Moirae')]
[('s', 'http://www.wikidata.org/entity/Q191192'), ('SName', 'Giants')]
[('s', 'http://www.wikidata.org/entity/Q578584'), ('SName', 'Oneiroi')]
[('s', 'http://www.wikidata.org/e

In [9]:
#Let's explore the instances of the above found "twelve olympians" class

queryString = """
SELECT DISTINCT ?s ?SName
WHERE {
   ?s wdt:P31 wd:Q101609. #a TwelveOlympians
   
   ?s sc:name ?SName
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q41680'), ('SName', 'Dionysus')]
1


In [11]:
#Weird result above (should be 12 instances, not 1), let's check properties/objects related to twelve olympians

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname
WHERE {
   wd:Q101609 ?p ?obj. #TwelveOlympians

   ?p sc:name ?Pname.
   ?obj sc:name ?Oname.

}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('Pname', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q20830276'), ('Oname', 'group of mythical characters')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('Pname', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q37340'), ('Oname', 'Apollo')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('Pname', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q37122'), ('Oname', 'Athena')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('Pname', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q38012'), ('Oname', 'Hera')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('Pname', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q39503'), ('Oname', 'Artemis')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('Pname', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q44384'), ('Oname', 'Hephaestus')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('Pname', 'has part'), ('obj', 'ht

In [61]:
#We see that we can access them thanks to the haspart property

queryString = """
SELECT DISTINCT ?Olympian ?Oname
WHERE {
   wd:Q101609 wdt:P527 ?Olympian. #TwelveOlympians hasPart
   
   ?Olympian sc:name ?Oname. 

}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

Results
[('Olympian', 'http://www.wikidata.org/entity/Q37340'), ('Oname', 'Apollo')]
[('Olympian', 'http://www.wikidata.org/entity/Q37122'), ('Oname', 'Athena')]
[('Olympian', 'http://www.wikidata.org/entity/Q38012'), ('Oname', 'Hera')]
[('Olympian', 'http://www.wikidata.org/entity/Q39503'), ('Oname', 'Artemis')]
[('Olympian', 'http://www.wikidata.org/entity/Q44384'), ('Oname', 'Hephaestus')]
[('Olympian', 'http://www.wikidata.org/entity/Q41484'), ('Oname', 'Hermes')]
[('Olympian', 'http://www.wikidata.org/entity/Q41127'), ('Oname', 'Poseidon')]
[('Olympian', 'http://www.wikidata.org/entity/Q41419'), ('Oname', 'Hestia')]
[('Olympian', 'http://www.wikidata.org/entity/Q34201'), ('Oname', 'Zeus')]
[('Olympian', 'http://www.wikidata.org/entity/Q35500'), ('Oname', 'Aphrodite')]
[('Olympian', 'http://www.wikidata.org/entity/Q40730'), ('Oname', 'Demeter')]
[('Olympian', 'http://www.wikidata.org/entity/Q40901'), ('Oname', 'Ares')]
12


Final query for this task

In [62]:
# Final Query

queryString = """
SELECT DISTINCT ?Olympian ?Oname
WHERE {
   wd:Q101609 wdt:P527 ?Olympian. #TwelveOlympians hasPart
   
   ?Olympian sc:name ?Oname. 

}
LIMIT 50
"""
print("Results")
x=run_query(queryString)

#We can note that the only instance of Twelve Olympians found above (Dionysus) is not in the list. 
#There's indeed a debate about who is the 12th olympian, between Dionysus and Hestia 

Results
[('Olympian', 'http://www.wikidata.org/entity/Q37340'), ('Oname', 'Apollo')]
[('Olympian', 'http://www.wikidata.org/entity/Q37122'), ('Oname', 'Athena')]
[('Olympian', 'http://www.wikidata.org/entity/Q38012'), ('Oname', 'Hera')]
[('Olympian', 'http://www.wikidata.org/entity/Q39503'), ('Oname', 'Artemis')]
[('Olympian', 'http://www.wikidata.org/entity/Q44384'), ('Oname', 'Hephaestus')]
[('Olympian', 'http://www.wikidata.org/entity/Q41484'), ('Oname', 'Hermes')]
[('Olympian', 'http://www.wikidata.org/entity/Q41127'), ('Oname', 'Poseidon')]
[('Olympian', 'http://www.wikidata.org/entity/Q41419'), ('Oname', 'Hestia')]
[('Olympian', 'http://www.wikidata.org/entity/Q34201'), ('Oname', 'Zeus')]
[('Olympian', 'http://www.wikidata.org/entity/Q35500'), ('Oname', 'Aphrodite')]
[('Olympian', 'http://www.wikidata.org/entity/Q40730'), ('Oname', 'Demeter')]
[('Olympian', 'http://www.wikidata.org/entity/Q40901'), ('Oname', 'Ares')]
12


# Task 8
Compare ancient Greece with ancient Egypt and find how many deities each one has.

In [4]:
#We need to find a way to link deities and ancient civilizations
#Let's see the properties/objects related to deities

queryString = """
SELECT DISTINCT ?obj ?Oname
WHERE {
   wd:Q178885 ?p ?obj. #deity

   ?obj sc:name ?Oname.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5648972'), ('Oname', 'Category:Deities')]
[('obj', 'http://www.wikidata.org/entity/Q106727050'), ('Oname', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('obj', 'http://www.wikidata.org/entity/Q190'), ('Oname', 'God')]
[('obj', 'http://www.wikidata.org/entity/Q105890973'), ('Oname', 'supernatural realm')]
[('obj', 'http://www.wikidata.org/entity/Q17624054'), ('Oname', 'fictional deity')]
[('obj', 'http://www.wikidata.org/entity/Q4271324'), ('Oname', 'mythical character')]
[('obj', 'http://www.wikidata.org/entity/Q15619164'), ('Oname', 'abstract being')]
[('obj', 'http://www.wikidata.org/entity/Q28855038'), ('Oname', 'supernatural being')]
[('obj', 'http://www.wikidata.org/entity/P551'), ('Oname', 'residence')]
[('obj', 'http://www.wikidata.org/entity/P460'), ('Oname', 'said to be the same as')]
[('obj', 'http://www.wikidata.org/entity/Q1379405'), ('Oname', 'divinity')]
[('obj', 'http://www.wikidata.org/entity/Q10809670'), ('Oname',

In [13]:
#let's see the type of the objects linked to "worshipped by" property

queryString = """
SELECT DISTINCT ?worshType ?Wname
WHERE {
   ?s wdt:P1049 ?worshipper. #worshippedBy
   ?worshipper wdt:P31 ?worshType. #a

   ?worshType sc:name ?Wname.

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('worshType', 'http://www.wikidata.org/entity/Q1189816'), ('Wname', 'ethnic religion')]
[('worshType', 'http://www.wikidata.org/entity/Q24286894'), ('Wname', 'fictional humanoid species')]
[('worshType', 'http://www.wikidata.org/entity/Q2590445'), ('Wname', 'fictional ethnic group')]
[('worshType', 'http://www.wikidata.org/entity/Q73756872'), ('Wname', 'Star Trek species')]
[('worshType', 'http://www.wikidata.org/entity/Q1649443'), ('Wname', 'national Church')]
[('worshType', 'http://www.wikidata.org/entity/Q2325038'), ('Wname', 'state church')]
[('worshType', 'http://www.wikidata.org/entity/Q32090'), ('Wname', 'lifestyle')]
[('worshType', 'http://www.wikidata.org/entity/Q3586152'), ('Wname', 'Episcopal Church')]
[('worshType', 'http://www.wikidata.org/entity/Q2728698'), ('Wname', 'religious belief')]
[('worshType', 'http://www.wikidata.org/entity/Q28171280'), ('Wname', 'ancient civilization')]
[('worshType', 'http://www.wikidata.org/entity/Q879146'), ('Wname', 'Christian deno

In [11]:
#Note that we already found Ancient greece BGP while getting instances of ancient civilizations: [('s', 'http://www.wikidata.org/entity/Q11772'), ('CountryName', 'Ancient Greece')]
#Let's check if worshipped by returns everything for our 2 desired civilizations :

queryString = """
SELECT DISTINCT ?c ?Cname ?deity ?Dname
WHERE {
    VALUES ?c {wd:Q11772 wd:Q11768} #AncientGreece AncientEgypt
    
    ?deity wdt:P1049 ?c. #worshippedBy

   ?c sc:name ?Cname.
   ?deity sc:name ?Dname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q11768'), ('Cname', 'Ancient Egypt'), ('deity', 'http://www.wikidata.org/entity/Q129106'), ('Dname', 'Bastet')]
[('c', 'http://www.wikidata.org/entity/Q11768'), ('Cname', 'Ancient Egypt'), ('deity', 'http://www.wikidata.org/entity/Q146921'), ('Dname', 'Thoth')]
[('c', 'http://www.wikidata.org/entity/Q11772'), ('Cname', 'Ancient Greece'), ('deity', 'http://www.wikidata.org/entity/Q214987'), ('Dname', 'Eirene')]
[('c', 'http://www.wikidata.org/entity/Q11768'), ('Cname', 'Ancient Egypt'), ('deity', 'http://www.wikidata.org/entity/Q4066410'), ('Dname', 'Anit')]
4


In [15]:
#It seems very incomplete, let's change the strategy
#Let's see who worships Poseidon (bgp is given)

queryString = """
SELECT DISTINCT ?c ?Cname ?deity ?Dname
WHERE {
    VALUES ?deity {wd:Q41127} #Poseidon
    
    ?deity wdt:P1049 ?c. #worshippedBy

   ?c sc:name ?Cname.
   ?deity sc:name ?Dname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)
#It is Greek mytology and not Ancient Greece.

Results
[('c', 'http://www.wikidata.org/entity/Q34726'), ('Cname', 'Greek mythology'), ('deity', 'http://www.wikidata.org/entity/Q41127'), ('Dname', 'Poseidon')]
1


In [21]:
#Let's find out if there is an equivalent for Ancient Egypt (like egyptian mythology or Ancient egypt mythology)
#To do so, we have to find an ancient Egypt deity
#Let's get back to Ancient Egypt properties/objects

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname
WHERE {
   wd:Q11768 ?p ?obj. #AncientEgypt

   ?p sc:name ?Pname.
   ?obj sc:name ?Oname.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)


Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('Pname', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q28171280'), ('Oname', 'ancient civilization')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('Pname', 'history of topic'), ('obj', 'http://www.wikidata.org/entity/Q766876'), ('Oname', 'history of ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('Pname', 'follows'), ('obj', 'http://www.wikidata.org/entity/Q714601'), ('Oname', 'Predynastic Period of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q1138524'), ('Oname', 'Paulys Realenzyklopädie der klassischen Altertumswissenschaft')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('Pname', 'continent'), ('obj', 'http://www.wikidata.org/entity/Q15'), ('Oname', 'Africa')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q

In [22]:
#Nothing promissing, let's use a result from a previous query : [('obj', 'http://www.wikidata.org/entity/Q145903'), ('Oname', 'egyptology')]
#Checking properties/objects linked

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname
WHERE {
   wd:Q145903 ?p ?obj. #egyptology

   ?p sc:name ?Pname.
   ?obj sc:name ?Oname.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2578'), ('Pname', 'studies'), ('obj', 'http://www.wikidata.org/entity/Q11768'), ('Oname', 'Ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('Pname', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q11862829'), ('Oname', 'academic discipline')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q19180675'), ('Oname', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q2657718'), ('Oname', 'Armenian Soviet Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q3181656'), ('Oname', 'The Nuttall Encyclopædia')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('Pname', 'instance of'), ('obj', 'http://www.wikidata.org/e

In [24]:
#We found the Bgp for Egyptian Mythology. ('obj', 'http://www.wikidata.org/entity/Q205740'), ('Oname', 'Egyptian mythology')]
#Let's explore it

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname
WHERE {
   wd:Q205740 ?p ?obj. #EgyptianMythology

   ?p sc:name ?Pname.
   ?obj sc:name ?Oname.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('Pname', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q145915'), ('Oname', 'Nekhbet')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('Pname', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q9174'), ('Oname', 'religion')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('Pname', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q1047113'), ('Oname', 'specialty')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('Pname', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q447131'), ('Oname', 'religion of ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('Pname', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q9134'), ('Oname', 'mythology')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('Pname', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q9134'), ('Oname', 'mythology')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('Pname', 'part of'), (

In [27]:
#I forgot to explore greek mythology :

queryString = """
SELECT DISTINCT ?p ?Pname ?obj ?Oname
WHERE {
   wd:Q34726 ?p ?obj. #GreekMythology

   ?p sc:name ?Pname.
   ?obj sc:name ?Oname.

}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q106727050'), ('Oname', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q1138524'), ('Oname', 'Paulys Realenzyklopädie der klassischen Altertumswissenschaft')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q19180675'), ('Oname', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('Pname', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q602358'), ('Oname', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('Pname', 'part of'), ('obj', 'http://www.wikidata.org/entity/Q1367434'), ('Oname', 'classical mythology')]
[('p', 'http://w

In [63]:
#We can list all deities worshipped by both mythologies now (using the Values keyword)

queryString = """
SELECT DISTINCT ?c ?Cname ?deity ?Dname
WHERE {
    VALUES ?c {wd:Q34726 wd:Q205740} #GreekMyth EgyptianMyth
    
    ?deity wdt:P1049 ?c. #worshippedBy

   ?c sc:name ?Cname.
   ?deity sc:name ?Dname.
}
GROUP BY ?c ?Cname ?deity ?Dname
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q34726'), ('Cname', 'Greek mythology'), ('deity', 'http://www.wikidata.org/entity/Q180325'), ('Dname', 'Ersa')]
[('c', 'http://www.wikidata.org/entity/Q34726'), ('Cname', 'Greek mythology'), ('deity', 'http://www.wikidata.org/entity/Q193115'), ('Dname', 'Scylla')]
[('c', 'http://www.wikidata.org/entity/Q34726'), ('Cname', 'Greek mythology'), ('deity', 'http://www.wikidata.org/entity/Q183417'), ('Dname', 'Chiron')]
[('c', 'http://www.wikidata.org/entity/Q34726'), ('Cname', 'Greek mythology'), ('deity', 'http://www.wikidata.org/entity/Q192407'), ('Dname', 'Orestes')]
[('c', 'http://www.wikidata.org/entity/Q34726'), ('Cname', 'Greek mythology'), ('deity', 'http://www.wikidata.org/entity/Q617977'), ('Dname', 'Apate')]
[('c', 'http://www.wikidata.org/entity/Q34726'), ('Cname', 'Greek mythology'), ('deity', 'http://www.wikidata.org/entity/Q1235379'), ('Dname', 'Pasithea')]
[('c', 'http://www.wikidata.org/entity/Q34726'), ('Cname', 'Greek mytholo

In [64]:
#Attempt 1 : adding the aggregate

queryString = """
SELECT DISTINCT ?Cname (count(?deity) as ?count)
WHERE {
    VALUES ?c {wd:Q34726 wd:Q205740} #GreekMyth EgyptianMyth
    
    ?deity wdt:P1049 ?c. #worshippedBy

   ?c sc:name ?Cname.
   ?deity sc:name ?Dname.
}
GROUP BY ?Cname
LIMIT 30
"""

print("Results")
x=run_query(queryString)

Results
[('Cname', 'Egyptian mythology'), ('count', '19')]
[('Cname', 'Greek mythology'), ('count', '99')]
2


In [7]:
#19 egyptian deities seems to be a low amount
#Let's try another approach to see if we get similar result
#We know greek deity is a subclass of deity, let's see if there is an equivalent for ancient egypt deities using regex

queryString = """
SELECT DISTINCT ?deities ?Dname
WHERE {

    ?deities wdt:P279 wd:Q178885 . #subclassOf deity

    ?deities sc:name ?Dname.
    FILTER(REGEX(?Dname,"([Ee]gypt|[Gg]ree)"))
}
"""

print("Results")
x=run_query(queryString)

Results
[('deities', 'http://www.wikidata.org/entity/Q146083'), ('Dname', 'Ancient Egyptian deity')]
[('deities', 'http://www.wikidata.org/entity/Q22989102'), ('Dname', 'Greek deity')]
2


In [25]:
#Let's use the two bgp found above and count their instances

queryString = """
SELECT (count(distinct ?egyptianDeity) as ?AncienEgyptDeities) (count(distinct ?greekDeity) as ?AncientGreeceDeities) 
WHERE {
    ?egyptianDeity wdt:P31 wd:Q146083. #AncientEgyptDeity
    ?greekDeity wdt:P31 wd:Q22989102. #GreekDeity
}
"""

print("Results")
x=run_query(queryString)

#This result seem more appropriate than the first one, let's keep it as the final answer

Results
[('AncienEgyptDeities', '281'), ('AncientGreeceDeities', '252')]
1


Final query for this task

In [26]:
# Final query

queryString = """
SELECT (count(distinct ?egyptianDeity) as ?AncienEgyptDeities) (count(distinct ?greekDeity) as ?AncientGreeceDeities) 
WHERE {
    ?egyptianDeity wdt:P31 wd:Q146083. #a AncientEgyptDeity
    ?greekDeity wdt:P31 wd:Q22989102. #a GreekDeity
}
"""

print("Results")
x=run_query(queryString)

Results
[('AncienEgyptDeities', '281'), ('AncientGreeceDeities', '252')]
1
